In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from scipy.stats import pearsonr, spearmanr
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from SentenceComparison.SentenceComparison import SentenceComparison
from ModelHelpers.fill_mask_helpers import get_probability_from_pipeline_for_token
from relation_templates.templates import relations, nominalized_relations, get_length_for_relation, get_templates, get_relation_meta, get_relation_name, get_order_for_relation, get_distance_for_relation

In [3]:
from TypologyQuerier import TypologyQuerier 

In [4]:
logging.set_verbosity_error()

In [5]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(model(sentence), token)
    return prob

In [6]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]
TOP_K = 10

In [7]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
#relations = nominalized_relations

In [8]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
my_tokenizer = lambda sentence: tokenizer(sentence)['input_ids']

In [10]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [11]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 28971.00it/s]


In [12]:
unmasker.save_to_cache()

In [13]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=500)

In [14]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(unmasker(sentence), token)
    return prob

In [15]:
Comparer = SentenceComparison(relations, get_templates, metric, MASK, get_relation_meta)
Comparer.compare(TREx.data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 2397.75it/s]


In [16]:
unmasker.save_to_cache()

In [17]:
accuracies = querier.results_for_calculation()
np_accuracies = np.array(accuracies)[1:,1:].flatten().astype(float)

In [18]:
probabilities = Comparer.make_global_comparison()
np_probabilities = np.array(probabilities)[:,1:].flatten().astype(float)

In [19]:
lengths = get_distance_for_relation(my_tokenizer)
np_lengths = np.array(lengths)[1:,1:].flatten().astype(float)

Accuracy vs Length

In [20]:
r_corr , _ = pearsonr(np_accuracies, np_lengths)
r_corr

-0.06261672153790084

In [21]:
r_corr , _ = spearmanr(np_accuracies, np_lengths)
r_corr

-0.05407570437055938

Probability vs Length

In [22]:
r_corr , _ = pearsonr(np_probabilities, np_lengths)
r_corr, r_corr**2*100

(-0.08701967494654411, 0.7572423827802197)

In [23]:
r_corr , _ = spearmanr(np_probabilities, np_lengths)
r_corr, r_corr**2*100

(-0.2267665489565759, 5.142306772567513)

In [24]:
for row in lengths:
    print(" & ".join(map(str, row))+" \\\\")

relations & simple & compound & complex & compound-complex \\
P159 & 3 & 4 & 4 & 5 \\
P37 & 2 & 4 & 4 & 9 \\
P1412 & 5 & 5 & 5 & 12 \\
P138 & 4 & 2 & 2 & 2 \\
P495 & 4 & 5 & 5 & 10 \\
P17 & 4 & 5 & 4 & 14 \\
P178 & 4 & 12 & 3 & 17 \\
P103 & 2 & 2 & 8 & 2 \\
P20 & 3 & 8 & 9 & 15 \\
P36 & 2 & 10 & 10 & 14 \\
P407 & 4 & 10 & 10 & 18 \\
P190 & 2 & 2 & 2 & 2 \\
P276 & 4 & 9 & 8 & 13 \\
P108 & 3 & 9 & 7 & 5 \\
P31 & 3 & 11 & 9 & 10 \\
P364 & 2 & 11 & 10 & 13 \\
P27 & 2 & 5 & 6 & 11 \\
P30 & 4 & 5 & 6 & 11 \\
P47 & 4 & 10 & 11 & 2 \\
P1376 & 5 & 11 & 9 & 14 \\
P106 & 3 & 13 & 7 & 22 \\
P413 & 3 & 5 & 5 & 13 \\
P937 & 5 & 5 & 5 & 10 \\
P449 & 5 & 5 & 6 & 14 \\
P131 & 4 & 5 & 6 & 13 \\
P1303 & 2 & 8 & 12 & 12 \\
P127 & 4 & 8 & 6 & 5 \\
P279 & 7 & 7 & 11 & 13 \\
P740 & 4 & 5 & 5 & 10 \\
P19 & 4 & 5 & 5 & 15 \\
P527 & 3 & 4 & 8 & 4 \\
P136 & 2 & 5 & 4 & 13 \\
P264 & 6 & 8 & 2 & 14 \\
P1001 & 6 & 5 & 10 & 3 \\
P39 & 5 & 9 & 8 & 13 \\
P101 & 6 & 13 & 9 & 18 \\
P140 & 7 & 9 & 4 & 10 \\
P361 & 4 & 10

### simple vs ...

In [25]:
keys = ["Compound", "Complex", "Compound-Complex"]
key_accuracies = {key:np.array(accuracies)[1:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}
key_probabilities = {key:np.array(probabilities)[:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}
key_lengths = {key:np.array(lengths)[1:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}

In [26]:
for c_key in keys:
    r_ac, _ = pearsonr(key_accuracies[c_key], key_lengths[c_key])
    r_prob, _ = pearsonr(key_probabilities[c_key], key_lengths[c_key])
    print(f"Simple vs {c_key} - accuracy: {r_ac} - probability: {r_prob}")

Simple vs Compound - accuracy: -0.1795171922908547 - probability: -0.07864581206275142
Simple vs Complex - accuracy: -0.21470828640608436 - probability: -0.03604651720297235
Simple vs Compound-Complex - accuracy: -0.13425172974996524 - probability: -0.1851206527291276


In [27]:
for c_key in keys:
    r_ac, _ = spearmanr(key_accuracies[c_key], key_lengths[c_key])
    r_prob, _ = spearmanr(key_probabilities[c_key], key_lengths[c_key])
    print(f"Simple vs {c_key} - accuracy: {r_ac} - probability: {r_prob}")

Simple vs Compound - accuracy: -0.18726027885747387 - probability: -0.24177611674551158
Simple vs Complex - accuracy: -0.1951768539243625 - probability: -0.10724008747661005
Simple vs Compound-Complex - accuracy: -0.1090617369445727 - probability: -0.26717367113256607


In [28]:
for c_key in keys:
    r_ac, _ = pearsonr(key_accuracies[c_key], key_lengths[c_key])
    s_ac, _ = spearmanr(key_accuracies[c_key], key_lengths[c_key])
    print(f"10 & Simple and {c_key} & {r_ac:.4f} & {s_ac:.4f} \\\\")

10 & Simple and Compound & -0.1795 & -0.1873 \\
10 & Simple and Complex & -0.2147 & -0.1952 \\
10 & Simple and Compound-Complex & -0.1343 & -0.1091 \\


In [29]:
my_tokenizer("you are dumb")

[101, 1128, 1132, 14908, 102]

In [30]:
my_tokenizer("you")

[101, 1128, 102]

In [31]:
my_tokenizer("fddfsdf")

[101, 175, 13976, 22816, 1181, 2087, 102]

In [32]:
my_tokenizer("subject")

[101, 2548, 102]

In [33]:
my_tokenizer("object")

[101, 4231, 102]